In [2]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds


In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 1000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [6]:
BATCH_SIZE = 1000

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape, validation_targets.shape)

(6000, 28, 28, 1) (6000,)


In [7]:
input_size = 784
output_size = 10

hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
NUM_EPOCHS = 30

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data, epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=1)


Epoch 1/30
54/54 - 8s - 157ms/step - accuracy: 0.7056 - loss: 1.0952 - val_accuracy: 0.8773 - val_loss: 0.4350
Epoch 2/30
54/54 - 6s - 105ms/step - accuracy: 0.9015 - loss: 0.3534 - val_accuracy: 0.9158 - val_loss: 0.2993
Epoch 3/30
54/54 - 4s - 69ms/step - accuracy: 0.9218 - loss: 0.2784 - val_accuracy: 0.9278 - val_loss: 0.2544
Epoch 4/30
54/54 - 3s - 47ms/step - accuracy: 0.9317 - loss: 0.2406 - val_accuracy: 0.9347 - val_loss: 0.2262
Epoch 5/30
54/54 - 3s - 50ms/step - accuracy: 0.9398 - loss: 0.2122 - val_accuracy: 0.9400 - val_loss: 0.2053
Epoch 6/30
54/54 - 3s - 51ms/step - accuracy: 0.9470 - loss: 0.1878 - val_accuracy: 0.9453 - val_loss: 0.1865
Epoch 7/30
54/54 - 6s - 105ms/step - accuracy: 0.9524 - loss: 0.1683 - val_accuracy: 0.9483 - val_loss: 0.1717
Epoch 8/30
54/54 - 5s - 100ms/step - accuracy: 0.9571 - loss: 0.1516 - val_accuracy: 0.9518 - val_loss: 0.1600
Epoch 9/30
54/54 - 5s - 95ms/step - accuracy: 0.9602 - loss: 0.1393 - val_accuracy: 0.9552 - val_loss: 0.1526
Epoch 

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 817ms/step - accuracy: 0.9716 - loss: 0.0952
Test loss: 0.10. Test accuracy: 97.16%


In [20]:

# Ładujemy obrazek
image = cv2.imread("my_digit.png", cv2.IMREAD_GRAYSCALE)

image_resized = cv2.resize(image, (28, 28), interpolation=cv2.INTER_AREA)

# Odwracamy kolory (czarne tło → białe tło, biała cyfra → czarna)
image_inverted = 255 - image_resized

# Normalizacja i reshaping
image_scaled = image_inverted / 255.0
image_reshaped = image_scaled.reshape(1, 28, 28, 1)

# Przewidujemy
prediction = model.predict(image_reshaped)
predicted_label = np.argmax(prediction)

print(f"Model mówi: {predicted_label} 🎉")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Model mówi: 2 🎉
